<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/DeepLearningNLP/nlp201931195.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어처리 기말고사


**본 코드는 네이버의 영화 리뷰 데이터를 기반으로 이 리뷰가 긍정인지 부정인지를 예측하는 모델을 작성하는 것입니다.<br> 본 코드는 기본적인 모델로만 학습하여 예측을 하는 코드이며, CNN 및 RNN 모델을 생성하여 모델의 정확도를 높여야 합니다.**

아래와 같은 사항을 수정하여 기말고사의 점수를 획득할 수 있습니다.


1. 데이터 전처리 함수 (+10)
1. 데이터 토큰화 방법 (+10)
1. 모델 수정(+20)
1. 모델에 관한 설명(+20)
1. 테스트 정확도가 제시한 코드보다 높을 경우(+20) 테스트 정확도 ---- 0.8023
1. 테스트 정확도의 상대평가 (+20)


유의 사항 <br>

* 반드시 ipynb 확장자로 제출할 것
* 타인과 수정된 코드 공유 금지


In [6]:
!pip install konlpy #konlpy 설치

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 450kB 36.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [7]:
import numpy as np
import pandas as pd
import os
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


**학습데이터셋을 ratings_train 하나만 가지고 옴**

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt") # ratings_train.txt 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")# ratings_test.txt 테스트 데이터 다운로드 

('ratings_test.txt', <http.client.HTTPMessage at 0x7fce64d9c550>)

**학습데이터 확인해 보기**

In [9]:
train_data = pd.read_csv('ratings_train.txt', header=0, delimiter='\t', quoting=3)
train_data.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [10]:
test_data = pd.read_csv('ratings_test.txt', header=0, delimiter='\t', quoting=3)
test_data.head(10)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


**전처리 함수**

(전처리 함수 수정시 +10)

# 불용어 목록 추가를 위한 라이브러리 선언

In [11]:
!pip install beautifulsoup4 #beautifulsoup4 설치

In [12]:
import requests
from bs4 import BeautifulSoup

In [25]:
r = requests.get("https://www.ranks.nl/stopwords/korean", verify=False)# 불용어로 정의된 문서호출
html = r.text 
soup = BeautifulSoup(html, 'html.parser') #html형식으로 만들기 
stop_words = []
for tr in soup.select("tbody td"): # Korean Stopwords 문서의 단어 html select
    for td in (str(tr).replace("<td>","").replace("</td>",'').split("<br/>")): #html 내용중 문법 제거
        stop_words.append(td)

print(stop_words)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

In [26]:
# 감탄사, 조사들은 제거할 수 있도록 정의
stop_words.extend(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']) #불용어 제거 확장
stop_words = set(stop_words)# 중복제거

In [27]:
okt=Okt()

def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    #  문장에서 한글만 뽑아냄
    review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", review)
    
    # okt 라이브러리로 단어의 어간별로 분리
    word_review = okt.morphs(review_text, stem=True)
    
    #  감탄사와 조사 제거
    if remove_stopwords:
        word_review = [token for token in word_review if token not in stop_words]
        
    return word_review

**학습 데이터 전처리 수행**


In [16]:
def clean_review(doc):
    return preprocessing(doc, okt, True, stop_words) if type(doc) == str else []#pandas clean_review 저장 함수

In [32]:
clean_train_review = []

for review in train_data['document']: # review에서 행하나씩 가져오기
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, True, stop_words)) #훈련데이타 전처리 수행
    else:
        clean_train_review.append([])

In [33]:
clean_test_review = []

for review in test_data['document']: # review에서 행하나씩 가져오기
    if type(review) == str:
        clean_test_review.append(preprocessing(review, okt, True, stop_words)) #테스트데이타 전처리 수행
    else:
        clean_test_review.append([])

**토큰화** <br>
토큰 방법을 수정시 + 10

In [36]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

# 전체 순서를 만들고
tokenizer.fit_on_texts(clean_train_review)# word학습
tokenizer.fit_on_texts(clean_test_review)

#  단어의 위치를 숫자로 표시
train_sequence = tokenizer.texts_to_sequences(clean_train_review)
test_sequence = tokenizer.texts_to_sequences(clean_test_review)

MAX_SEQUENCE_LENGTH = 30

# 최대 문장 길이는 8로 padding
train_inputs = pad_sequences(train_sequence, maxlen = MAX_SEQUENCE_LENGTH, padding='post', value = 0.1)
test_inputs = pad_sequences(test_sequence, maxlen = MAX_SEQUENCE_LENGTH, padding='post', value = 0.1)

#  train, test label 만들기
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

**모델 만들기**
<br> (모델 수정시 + 20)

<모델에 관한 설명을 자세하게 서술시 +20><br>


In [44]:
import tensorflow as tf
import os
# TPU전략 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.7.33.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.7.33.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [51]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447

#  데이터를 train과 eval로 분리
# from sklearn.model_selection import train_test_split
# input_train, input_eval, label_train, label_eval = train_test_split(train_inputs, train_labels, test_size=TEST_SPLIT, random_state=RNG_SEED)

# 테스트 데이터 추가
input_train, input_eval, label_train, label_eval = train_inputs, test_inputs, train_labels, test_labels

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, Conv1D, LSTM, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
with strategy.scope():
    model = Sequential()
    #  Word Embedding
    word_size = len(tokenizer.word_index)+1 # Tokenizer 전체 싸이즈 +1만큼 embedding unit 갯수 인자
    model.add(Embedding(word_size, 128, input_length=MAX_SEQUENCE_LENGTH))  #embedding layer, embedding input길이 지정 
    model.add(LSTM(128, return_sequences=True)) #기본 활성함수 tanh사용, 다음 layer에서 반환값 사용을 위해 return_sequences활성
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=.1, recurrent_dropout=.5))) # bidirectional로 순환힉습, lstm의 recurrent와 dropout지정
    model.add(Dropout(0.3)) # 학습의 효율을 위해 소실 발생 시키기 

    #  DNN Training, Activation은 relu
    model.add(Flatten()) #다음 레이어를 위해 flatten하게 펴주는 layer
    model.add(Dense(1, activation='sigmoid')) #최종 출력층
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) # 최적화함수는 RMSPROP, 이진분류로 소실 함수는 binary_crossentropy, 지표는 acc

In [52]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) #val_loss가 가장 작은 상태일때 4번동안 참다가 멈추게 하는 작업
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) #최고의 모델(weights)을 저장하는 기능,

In [53]:
history = model.fit(input_train, label_train, epochs=20, callbacks=[es, mc], batch_size=60, validation_data=(input_eval, label_eval), validation_split=0.2)

Epoch 1/20
2000/2000 [==============================] - 57s 22ms/step - loss: 0.3953 - acc: 0.8175 - val_loss: 0.3476 - val_acc: 0.8454

Epoch 00001: val_acc improved from -inf to 0.84540, saving model to best_model.h5
Epoch 2/20
2000/2000 [==============================] - 43s 21ms/step - loss: 0.3270 - acc: 0.8576 - val_loss: 0.3363 - val_acc: 0.8535

Epoch 00002: val_acc improved from 0.84540 to 0.85350, saving model to best_model.h5
Epoch 3/20
2000/2000 [==============================] - 43s 21ms/step - loss: 0.2976 - acc: 0.8742 - val_loss: 0.3325 - val_acc: 0.8559

Epoch 00003: val_acc improved from 0.85350 to 0.85590, saving model to best_model.h5
Epoch 4/20
2000/2000 [==============================] - 42s 21ms/step - loss: 0.2720 - acc: 0.8881 - val_loss: 0.3400 - val_acc: 0.8554

Epoch 00004: val_acc did not improve from 0.85590
Epoch 5/20
2000/2000 [==============================] - 43s 21ms/step - loss: 0.2487 - acc: 0.8991 - val_loss: 0.3496 - val_acc: 0.8499

Epoch 00005: 

테스트 정확도가 아래 보다 높을 경우 +20 <br>
테스트 정확도가 과제를 수행한 사람중에 상대평가 하여 +20 추가
<br> 예) 테스트 정확도가 가장 높을 경우 20점 만점
<br>                    두번째로 높을 경우 19점
<br>                    세번째로 높을 경우 18점

In [54]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(input_eval, label_eval)[1]))

1563/1563 [==============================] - 32s 20ms/step - loss: 0.3405 - acc: 0.8506

 테스트 정확도: 0.8506


In [55]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stop_words] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = MAX_SEQUENCE_LENGTH) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


In [56]:
sentiment_predict("이거 졸렸음")

56.41% 확률로 부정 리뷰입니다.

